In [1]:
import json
from sklearn.model_selection import train_test_split
import pandas as pd
import re


In [2]:
with open('data/with_titles_and_abstract.json', 'r') as fin:
    data = json.load(fin)

In [3]:
df = pd.DataFrame(data)

In [4]:
columns_to_drop = [col for col in df.columns if col not in ['title', 'abstract']]

df = df.drop(columns=columns_to_drop, axis=1)

In [5]:
df = df.sample(frac=0.01)

In [6]:
with open('data/data_subset.json', 'w') as f:
    json.dump(df.to_json(), f)
    
    

In [8]:
df.to_json('data/data_subset.json')

In [9]:
pd.read_json('data/data_subset.json')

,title,abstract
1457742,"Gauged Lepton Number, Dirac Neutrinos, Dark Ma...",Lepton number is promoted to an $U(1)_L$ gau...
968455,Excitation transfer from Second to First reson...,We present experimental investigation on the...
1424508,A Thorough View of Exact Inference in Graphs f...,Performing inference in graphs is a common t...
1977598,"Finite Temperature Field Theory of ""Extreme Bl...","We treat the model which describes ""extreme ..."
1416304,Exponential Ergodicity for Non-Dissipative McK...,"Under Lyapunov and monotone conditions, the ..."
...,...,...
139356,CMB Lensing Constraints on Dark Energy and Mod...,Weak gravitational lensing leaves a characte...
1864887,COSMOGRAIL: the COSmological MOnitoring of GRA...,Aims: The knowledge of the redshift of a len...
211607,Properties of the Planetary Caustic Perturbation,Just two of 10 extrasolar planets found by m...
227640,The Fluid/Gravity Correspondence: a new perspe...,This talk gives an overview of the recently-...


In [31]:

train_test_ratio = 0.9
train_valid_ratio = 7/9
df_full_train, df_test = train_test_split(df, train_size = train_test_ratio, random_state = 1)
df_train, df_valid = train_test_split(df_full_train, train_size = train_valid_ratio, random_state = 1)

In [32]:
import random

def build_dataset(df, dest_path):
    f = open(dest_path, 'w')
    data = ''
    abstracts = df['abstract'].tolist()
    titles = df['title'].tolist()
    datapoints = [titles[i]+': '+abstracts[i] for i in range(len(titles))]
    for item in datapoints:
        if random.random() < 1:
            item = str(item).strip()
            item = re.sub(r"\s", " ", item)
            bos_token = '<|endoftext|>'
            eos_token = '<|endoftext|>'
            data += bos_token + ' ' + item + ' ' + eos_token + '\n'
        
    f.write(data)


In [33]:
build_dataset(df_train, 'data/gpt2_train.txt')
build_dataset(df_valid, 'data/gpt2_valid.txt')
build_dataset(df_test, 'data/gpt2_test.txt')

In [34]:
df.columns

Index(['title', 'abstract'], dtype='object')

In [35]:
df_train.to_json('data/train_small_best.json')
df_test.to_json('data/test_small_best.json')
df_valid.to_json('data/valid_small_best.json')

In [36]:
def build_json_dataset(df, path):
    with open(path, 'w') as fout:
        for i in df['title'].keys():
            item = {'title':df['title'][i], 'abstract':df['abstract'][i]}
            fout.write(json.dumps(item)+'\n')

In [37]:
build_json_dataset(df_train, 'data/train_small_best.json')
build_json_dataset(df_valid, 'data/valid_small_best.json')
build_json_dataset(df_test, 'data/test_small_best.json')

In [38]:
json.dumps({'title':df_train['title'][list(df_train['title'].keys())[0]], 'abstract':df_train['abstract'][list(df_train['title'].keys())[0]]})

'{"title": "Bulges", "abstract": "  We model the evolution of the galactic bulge and of the bulges of a selected\\nsample of external spiral galaxies, via the multiphase multizone evolution\\nmodel. We address a few questions concerning the role of the bulges within\\ngalactic evolution schemes and the properties of bulge stellar populations. We\\nprovide solutions to the problems of chemical abundances and spectral indices,\\nthe two main observational constraints to bulge structure.\\n"}'

In [ ]:
df_train

In [ ]:
d = df_train.to_dict()

In [ ]:
list(d['title'].keys()) == list(d['abstract'].keys())

In [ ]:
d['abstract']

In [ ]:
from datasets import load_dataset

train_set = load_dataset('json', data_files='data/train_small.json')

In [ ]:
train_set